In [46]:
# community_analysis.ipynb

# 📌 SECTION 1: IMPORTS
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from scipy.stats import chi2_contingency, kruskal
import os
import warnings
import json
warnings.filterwarnings("ignore")

In [47]:
base_path = "../../figures/graphs"

In [48]:
tags = {
    0: "Early Failure with Death",
    1: "Early Failure with Survival",
    2: "Intermediate Failure with Survival",
    3: "Sustained Survival"
}

def get_community_mapping(algo):
    community_mapping = {}
    community_df = pd.read_csv(os.path.join('../data/full', algo, "community_assignments.csv"))
    for community in community_df["communityId"].unique():
        community_nodes = community_df[community_df["communityId"] == community]["nodeId"].tolist()
        if(algo == 'reverse_hybrid'):
            label = f"Sub-community {community}"
        else:
            label = tags[community]

        community_mapping[str(community)] = {
            "label": label,
            "nodes": len(community_nodes)
        }
    return community_mapping

In [49]:
import xml.etree.ElementTree as ET

def expand_svg_canvas(input_svg_path, output_svg_path, extra_width=400):
    # Parse the SVG file
    tree = ET.parse(input_svg_path)
    root = tree.getroot()

    # Get the current width, height, and viewBox
    width = root.get('width')
    height = root.get('height')
    viewBox = root.get('viewBox')

    # Convert width and height to float if they have units
    def parse_dimension(value):
        return float(value.replace('px', '')) if value.endswith('px') else float(value)

    width_val = parse_dimension(width)
    height_val = parse_dimension(height)

    # Update width and viewBox
    new_width = width_val + extra_width
    root.set('width', str(new_width))
    if viewBox:
        vb_values = list(map(float, viewBox.split()))
        vb_values[2] += extra_width
        root.set('viewBox', ' '.join(map(str, vb_values)))

    # Save the modified SVG
    tree.write(output_svg_path)
    print(f"Canvas expanded and saved to {output_svg_path}")

In [50]:
import xml.etree.ElementTree as ET

def annotate_svg_with_ellipses_and_legend(input_svg_path, output_svg_path, community_mapping, ellipse_params):
    """
    Annotates an SVG file with ellipses around communities and adds a legend.

    Parameters:
    - input_svg_path (str): Path to the input SVG file.
    - output_svg_path (str): Path to save the annotated SVG.
    - community_mapping (dict): Mapping of community IDs to labels and node counts.
    - ellipse_params (dict): Parameters for ellipses (cx, cy, rx, ry, rotation).
    - community_colors (dict): Mapping of community IDs to hex color codes.
    """
    tree = ET.parse(input_svg_path)
    root = tree.getroot()

    ns = {'svg': 'http://www.w3.org/2000/svg'}
    ET.register_namespace('', ns['svg'])

    # Add ellipses and labels
    for cid, params in ellipse_params.items():
        label = community_mapping[cid]["label"]
        nodes = community_mapping[cid]["nodes"]
        cx, cy, rx, ry, rot = params["cx"], params["cy"], params["rx"], params["ry"], params["rotation"]

        ellipse = ET.Element('ellipse', {
            'cx': str(cx),
            'cy': str(cy),
            'rx': str(rx),
            'ry': str(ry),
            'transform': f'rotate({rot},{cx},{cy})',
            'fill': 'none',
            'stroke': 'grey',
            'stroke-width': '8'
        })

        text = ET.Element('text', {
            'x': str(cx),
            'y': str(cy),
            'font-size': '35',
            'text-anchor': 'middle',
            'fill': 'black',
            'font-weight': 'bold'
        })
        text.text = f"{cid} (n={nodes})"

        root.append(ellipse)
        root.append(text)

    tree.write(output_svg_path)
    print(f"Annotated SVG saved as {output_svg_path}")


In [51]:
def add_level0_legend(input_svg_path, output_svg_path, community_mapping, community_colors):
    tree = ET.parse(input_svg_path)
    root = tree.getroot()

    ns = {'svg': 'http://www.w3.org/2000/svg'}
    ET.register_namespace('', ns['svg'])

    # Add legend
    legend_x = 700
    legend_y = -500
    legend_spacing = 50

    for i, (cid, color) in enumerate(community_colors.items()):
        y_offset = legend_y + i * legend_spacing

        rect = ET.Element('rect', {
            'x': str(legend_x),
            'y': str(y_offset),
            'width': '40',
            'height': '40',
            'fill': color,
            'stroke': 'black',
            'stroke-width': '1'
        })

        label = ET.Element('text', {
            'x': str(legend_x + 50),
            'y': str(y_offset + 30),
            'font-size': '30',
            'fill': 'black',
            'font-weight': 'bold'
        })

        if cid == '1&2':
            label.text = f"{cid}: Overlapping between 1 and 2 (n=49)"
        elif cid == '2&3':
            label.text = f"{cid}: Overlapping between 1 and 2 (n=5)"
        else:
            label.text = f"{cid}: {community_mapping[cid]['label']}"

        root.append(rect)
        root.append(label)

    tree.write(output_svg_path)
    print(f"SVG with legend saved as {output_svg_path}")

In [52]:
import xml.etree.ElementTree as ET

def add_level1_legend_grouped(input_svg_path, output_svg_path, community_mapping, community_colors, slpa_grouping):
    tree = ET.parse(input_svg_path)
    root = tree.getroot()

    ns = {'svg': 'http://www.w3.org/2000/svg'}
    ET.register_namespace('', ns['svg'])

    legend_x = 750
    legend_y = -500
    line_spacing = 50
    group_spacing = 30

    y_offset = legend_y

    for slpa_id, subcomms in slpa_grouping.items():
        # Add SLPA group heading
        heading = ET.Element('text', {
            'x': str(legend_x),
            'y': str(y_offset),
            'font-size': '35',
            'fill': 'black',
            'font-weight': 'bold'
        })
        heading.text = f"SLPA CommID {slpa_id}"
        root.append(heading)
        y_offset += line_spacing

        for cid in subcomms:
            color = community_colors[cid]
            label = community_mapping[cid]["label"]

            rect = ET.Element('rect', {
                'x': str(legend_x),
                'y': str(y_offset-20),
                'width': '40',
                'height': '40',
                'fill': color,
                'stroke': 'black',
                'stroke-width': '1'
            })

            text = ET.Element('text', {
                'x': str(legend_x + 50),
                'y': str(y_offset+10),
                'font-size': '30',
                'fill': 'black',
                'font-weight': 'bold'
            })
            text.text = f"{cid}: {label}"

            root.append(rect)
            root.append(text)

            y_offset += line_spacing

        y_offset += group_spacing  # extra space between SLPA groups

    tree.write(output_svg_path)
    print(f"Grouped legend added and saved to {output_svg_path}")

In [53]:
input_svg_path = os.path.join(base_path, "level1_old.svg")
expanded_svg_path = os.path.join(base_path, "level1_expanded.svg")
annotated_svg_path = os.path.join(base_path, "annotated_leiden_level1.svg")
output_svg_path = os.path.join(base_path, "level1.svg")

expand_svg_canvas(input_svg_path, expanded_svg_path, extra_width=400)

community_mapping = get_community_mapping(algo = "reverse_hybrid")

ellipse_params = {
    "0": {"cx": -390, "cy": 250, "rx": 200, "ry": 200, "rotation": 0},
    "1": {"cx": -100, "cy": 280, "rx": 100, "ry": 120, "rotation": 0},
    "2": {"cx": -100, "cy": 510, "rx": 200, "ry": 120, "rotation": 0},
    "4": {"cx": 360, "cy": 425, "rx": 125, "ry": 75, "rotation": 30},
    "3": {"cx": 200, "cy": 450, "rx": 125, "ry": 50, "rotation": 60},
    "5": {"cx": 240, "cy": 85, "rx": 150, "ry": 275, "rotation": 35},
    "6": {"cx": 450, "cy": 275, "rx": 175, "ry": 100, "rotation": 0},
    "7": {"cx": 540, "cy": -20, "rx": 125, "ry": 225, "rotation": 0},
    "8": {"cx": 80, "cy": -350, "rx": 300, "ry": 280, "rotation": 0},
    "9": {"cx": -390, "cy": -250, "rx": 210, "ry": 230, "rotation": 0}
}

community_colors = {
    "0": '#AAE49D',
    "1": '#D3D881',
    '2': '#90E6C8',
    '3': '#BCD1AE',
    '4': '#F6CA80',
    '5': '#B3D8E8',
    '6': '#FEB9B5',
    '7': '#E6C8B2',
    '8': '#F6C5E6',
    '9': '#FEC18D',
}

annotate_svg_with_ellipses_and_legend(
    expanded_svg_path,
    annotated_svg_path,
    community_mapping,
    ellipse_params,
)

slpa_grouping = {
    "0": ["0", "1", "2"],
    "1": ["3", "4"],
    "2": ["5", "6", "7"],
    "3": ["8", "9"]
}


add_level1_legend_grouped(annotated_svg_path, output_svg_path, community_mapping, community_colors, slpa_grouping)

os.remove(expanded_svg_path)
os.remove(annotated_svg_path)

Canvas expanded and saved to ../../figures/graphs/level1_expanded.svg
Annotated SVG saved as ../../figures/graphs/annotated_leiden_level1.svg
Grouped legend added and saved to ../../figures/graphs/level1.svg


In [54]:
input_svg_path = os.path.join(base_path, "level0_old.svg")
expanded_svg_path = os.path.join(base_path, "level0_expanded.svg")
annotated_svg_path = os.path.join(base_path, "annotated_leiden_level0.svg")
output_svg_path = os.path.join(base_path, "level0.svg")

expand_svg_canvas(input_svg_path, expanded_svg_path, extra_width=600)

community_mapping = get_community_mapping(algo = "w_slpa")

ellipse_params = {
    "3": {"cx": -75, "cy": -350, "rx": 575, "ry":325, "rotation": -10},
    "2": {"cx": 425, "cy": 275, "rx": 450, "ry": 225, "rotation": -50},
    "1": {"cx": 375, "cy": -25, "rx": 250, "ry": 75, "rotation": -35},
    "0": {"cx": -325, "cy": 325, "rx": 400, "ry": 250, "rotation": 45},
}

community_colors = {
    "0": '#FF6155',
    "1": '#FF8F26',
    '1&2': '#F4BB00',
    '2': '#FCEC02',
    '2&3': '#D9E701',
    '3': '#70AA01',
}

annotate_svg_with_ellipses_and_legend(
    expanded_svg_path,
    annotated_svg_path,
    community_mapping,
    ellipse_params,
)

add_level0_legend(annotated_svg_path, output_svg_path, community_mapping, community_colors)

os.remove(expanded_svg_path)
os.remove(annotated_svg_path)

Canvas expanded and saved to ../../figures/graphs/level0_expanded.svg
Annotated SVG saved as ../../figures/graphs/annotated_leiden_level0.svg
SVG with legend saved as ../../figures/graphs/level0.svg
